#**Abdus Samad (DHC-474)**
#**Task 3**
#Auto Tagging Support Tickets Using LLM

In [1]:
!pip install -q transformers sentence-transformers scikit-learn datasets pandas joblib

# -------------------------
# 1) IMPORTS
# -------------------------


In [2]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score
import joblib

# -------------------------
# 2) LOAD DATASET
# -------------------------

In [4]:

print("Loading dataset from Hugging Face...")
ds = load_dataset("Tobi-Bueck/customer-support-tickets")

df = pd.DataFrame(ds["train"])
print("Dataset Preview:")
display(df.head())

# Combine subject + body as text
df["text"] = df["subject"].fillna("") + " " + df["body"].fillna("")

# Collect tags from tag_1 ... tag_8 columns
tag_cols = [c for c in df.columns if c.startswith("tag_")]
df["tag_list"] = df[tag_cols].values.tolist()

# Clean None values
df["tag_list"] = df["tag_list"].apply(lambda tags: [t.lower() for t in tags if t not in [None, "None", "nan"]])

print(f"Total tickets: {len(df)}")
print("Sample tags:", df["tag_list"].head())
print("Unique tags:", sorted({t for tags in df.tag_list for t in tags}))


Loading dataset from Hugging Face...
Dataset Preview:


,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51.0,Security,Outage,Disruption,Data Breach,None,None,None,None
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51.0,Account,Disruption,Outage,IT,Tech Support,None,None,None
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51.0,Product,Feature,Tech Support,None,None,None,None,None
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51.0,Billing,Payment,Account,Documentation,Feedback,None,None,None
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51.0,Product,Feature,Feedback,Tech Support,None,None,None,None


Total tickets: 48587
Sample tags: 0          [security, outage, disruption, data breach]
1      [account, disruption, outage, it, tech support]
2                     [product, feature, tech support]
3    [billing, payment, account, documentation, fee...
4           [product, feature, feedback, tech support]
Name: tag_list, dtype: object
Unique tags: ['2019', 'abnormal', 'abonnement', 'abrechnungssystem', 'access', 'access control', 'access controls', 'access difficulty', 'access issue', 'access log', 'access management', 'access restriction', 'access-control', 'access_control', 'accesscontrol', 'accesscontrols', 'accessibility', 'accessmanagement', 'accessory', 'account', 'accounting', 'accrual', 'accrued', 'accuracy', 'action', 'actionteam', 'activation', 'activecampaign', 'ad', 'ad segment', 'ad spend', 'ad spending', 'adblocker', 'adblocking', 'adcontent', 'adcreative', 'add_ins', 'addins', 'address', 'adfatigue', 'adjustment', 'admin', 'adobe', 'adobe premiere pro', 'adobe sign', '

# -------------------------
# 3) PREPARE LABELS
# -------------------------

In [5]:
unique_labels = sorted({t for tags in df.tag_list for t in tags})
mlb = MultiLabelBinarizer(classes=unique_labels)
Y = mlb.fit_transform(df["tag_list"])

texts = df["text"].tolist()

# -------------------------
# 4) ZERO-SHOT CLASSIFIER
# -------------------------

In [7]:
print("\nLoading zero-shot pipeline...")
zero_shot = pipeline("zero-shot-classification",
                     model="facebook/bart-large-mnli",
                     device=0 if torch.cuda.is_available() else -1)

def zero_shot_predict(text, candidate_labels, top_k=3):
    res = zero_shot(text, candidate_labels, multi_label=True)
    labels = res["labels"]
    scores = res["scores"]
    ranked = list(zip(labels, scores))
    return ranked[:top_k]

print("\nZero-shot demo:")
print(zero_shot_predict(texts[0], unique_labels, top_k=3))


Loading zero-shot pipeline...


Device set to use cuda:0



Zero-shot demo:
[('infrastructure', 0.988297700881958), ('potential consequence', 0.980190634727478), ('alarm', 0.9800743460655212)]


# --------------------------------
# 5) FEW-SHOT k-NN ON EMBEDDINGS
# --------------------------------

In [8]:
print("\nLoading sentence-transformer embeddings...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embed_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)


support_texts, support_labels = [], []
for label in unique_labels:
    example = df[df["tag_list"].apply(lambda tags: label in tags)].head(1)
    if len(example) > 0:
        support_texts.append(example.iloc[0]["text"])
        support_labels.append(label)

support_embeddings = embed_model.encode(support_texts, convert_to_tensor=True)

def few_shot_predict(text, top_k=3):
    emb = embed_model.encode(text, convert_to_tensor=True)
    cos_scores = util.cos_sim(emb, support_embeddings)[0]
    label_scores = {support_labels[i]: float(cos_scores[i]) for i in range(len(support_labels))}
    ranked = sorted(label_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

print("\nFew-shot demo:")
print(few_shot_predict(texts[0], top_k=3))


Loading sentence-transformer embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1519 [00:00<?, ?it/s]


Few-shot demo:
[('data breach', 1.0), ('disruption', 1.0), ('outage', 1.0)]


# ------------------------------------------
# 6) SUPERVISED CLASSIFIER ON EMBEDDINGS
# ------------------------------------------

In [9]:
print("\nTraining supervised classifier (Logistic Regression)...")

X_train, X_test, y_train, y_test = train_test_split(
    embeddings.cpu().numpy(), Y, test_size=0.2, random_state=42
)

clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_train, y_train)

joblib.dump({"clf": clf, "mlb": mlb, "embed_model": "all-MiniLM-L6-v2"}, "ticket_tagging_model.joblib")

y_pred = clf.predict(X_test)
f1_micro = f1_score(y_test, y_pred, average="micro")
f1_macro = f1_score(y_test, y_pred, average="macro")

print(f"Supervised Evaluation -> F1-micro: {f1_micro:.4f}, F1-macro: {f1_macro:.4f}")


Training supervised classifier (Logistic Regression)...


/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 1 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 22 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 25 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 41 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 54 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 72 is present in all training exampl

Supervised Evaluation -> F1-micro: 0.6083, F1-macro: 0.0090


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# -------------------------
# 7) COMPARISON ON SAMPLE
# -------------------------

In [10]:
sample_text = "I cannot login to my account, password reset is not working."
print("\nSample Ticket:", sample_text)

print("Zero-shot:", zero_shot_predict(sample_text, unique_labels))
print("Few-shot:", few_shot_predict(sample_text))
emb = embed_model.encode(sample_text, convert_to_tensor=True).cpu().numpy().reshape(1, -1)
probas = clf.predict_proba(emb)[0]
sup_top = [mlb.classes_[i] for i in np.argsort(probas)[::-1][:3]]
print("Supervised (top-3):", sup_top)

print("\n✅ All steps completed. Model + predictions ready!")


Sample Ticket: I cannot login to my account, password reset is not working.
Zero-shot: [('unsuccessful', 0.9992147088050842), ('technical difficulty', 0.998298704624176), ('concern', 0.9980844855308533)]
Few-shot: [('unsuccessful', 0.6177732944488525), ('backlog', 0.5877218246459961), ('evening', 0.5877218246459961)]
Supervised (top-3): ['login', 'account', 'technical']

✅ All steps completed. Model + predictions ready!
